### 1. 필요한 라이브러리 import

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# 재현성을 위한 시드 설정
np.random.seed(42)
torch.manual_seed(42)

# device 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

### 2. 데이터 생성 및 전처리

#### 데이터 불러오기

In [ ]:
# 1. 가상 데이터 생성 (정상 데이터는 사인파, 이상치는 급격한 노이즈)
t = np.linspace(0, 100, 1000)
normal_signal = np.sin(t).reshape(-1, 1)

# Train: 정상 데이터만 존재
train_data = normal_signal[:800] 

# Test: 정상 데이터 + 강한 이상치(Outliers) 삽입
test_data = normal_signal[800:].copy()
test_data[50:60] += 10.0  # 강력한 이상치 신호

In [ ]:
# 1. 예시 데이터 생성
df = pd.DataFrame({
    '이름': ['A', 'B', 'C', 'A', 'E', 'F', 'G'],
    '점수': [10, 20, 30, 40, 50, 60, 70],
})

# 2. '이름' 열에서 'A'가 마지막으로 등장하는 인덱스 찾기
# (데이터에 'A'가 없을 경우를 대비해 안전하게 처리)
target_name = 'A'
indices = df[df['이름'] == target_name].index

if not indices.empty:
    # 마지막 인덱스 추출
    last_index = indices[-1]
    
    # 해당 위치부터 끝까지 슬라이싱
    df_result = df.iloc[last_index:]

    # 3. CSV 파일로 저장
    # index=False: 0, 1, 2 같은 행 번호는 파일에 저장하지 않음
    # encoding='utf-8-sig': 엑셀에서 열었을 때 한글 깨짐 방지
    df_result.to_csv('result_data.csv', index=False, encoding='utf-8-sig')
    
    print("저장이 완료되었습니다.")
    print(df_result)
else:
    print(f"'{target_name}'을(를) 찾을 수 없어 저장하지 못했습니다.")

💡 저장 옵션 설명
'result_data.csv': 저장할 파일 이름입니다. 원하는 경로가 있다면 'C:/folder/result.csv' 처럼 전체 경로를 적어주셔도 됩니다.

index=False: 데이터프레임 왼쪽에 붙는 숫자(0, 1, 2...)를 제외하고 실제 데이터만 저장합니다. 보통 결과물 파일에는 포함하지 않는 경우가 많습니다.

encoding='utf-8-sig': 특히 한국어 윈도우 환경의 엑셀에서 CSV 파일을 열 때 한글이 깨지는 것을 막아주는 가장 확실한 인코딩 방식입니다.

이제 이 코드를 실행하면 실행 경로에 result_data.csv 파일이 생성됩니다! 추가로 궁금한 점이 있으시면 말씀해 주세요.

In [ ]:

# HW state에서 볼 SVID
ctrl_hw = {'heater':[], 'gauge':[]}
nonctrl_hw = {}

# Process state에서 볼 SVID
ctrl_process = {} # step_state에 따라 학습할 set이 바뀜 (step을 지정하고 그 step에서만 학습함)
nonctrl_process = {} 


# df 중 지정한 SVID(컬럼명)중에서 step_state.value 같은 것 끼리 묶음
# 그럼 df를 2가지로 나눠야함 상시인거와 아닌거
# 그리고 나서 df중 df_state를 step_state.value별로 따로따로 생성

for i in df:
    # '소속' 열의 값이 같은 것끼리 묶어서 딕셔너리에 저장
    step = {name: group for name, group in df.groupby('step_no')}

   
# 확인하기
print(step['1'])


step_state = {
    9:[],
    10:[],
    11:[],
    12:[],
    13:[],
    14:[],
    15:[],
    16:[],
    17:[],
    18:[],
    19:[],
    20:[],
    21:[],
    22:[],
    23:[],
    24:[],
    25:[],
    26:[]
    }




In [1]:
import pandas as pd

# 예시 데이터프레임 생성
df = pd.DataFrame({
    '이름': ['Sora', 'Minho', 'Jane', 'Kevin', 'Lia'],
    '소속': ['A팀', 'B팀', 'A팀', 'C팀', 'B팀'],
    '점수': [90, 85, 95, 80, 70]
})

# '소속' 열의 값이 같은 것끼리 묶어서 딕셔너리에 저장
new_dfs = {name: group for name, group in df.groupby('소속')}

# 확인하기
print(new_dfs['A팀'])

     이름  소속  점수
0  Sora  A팀  90
2  Jane  A팀  95


In [4]:
from collections import defaultdict

data = {'A': 1, 'B': 2, 'C': 1, 'D': 3, 'E': 2}

new_data = defaultdict(list)
for key, value in data.items():
    new_data[value].append(key)

# 일반 딕셔너리로 변환 (필요한 경우)
result = dict(new_data)
print(result)
# 결과: {1: ['A', 'C'], 2: ['B', 'E'], 3: ['D']}

# P2, Zr, Hf, O3
#  0   1   2   3

{1: ['A', 'C'], 2: ['B', 'E'], 3: ['D']}


# Factory Method

class Pizza: pass
class CheesePizza(Pizza): pass
class PepperoniPizza(Pizza): pass

def create_pizza(kind):
    if kind == "cheese":
        return CheesePizza()
    elif kind == "pepperoni":
        return PepperoniPizza()
    else:
        return None

p = create_pizza("cheese")

# Abstract Factory

# 제품 인터페이스
class Chair: pass
class Table: pass

# 윈도우 스타일 제품
class WindowsChair(Chair): pass
class WindowsTable(Table): pass

# 맥 스타일 제품
class MacChair(Chair): pass
class MacTable(Table): pass

# 추상 팩토리 인터페이스
class UIFactory:
    def create_chair(self): pass
    def create_table(self): pass

# 구체 팩토리
class WindowsFactory(UIFactory):
    def create_chair(self): return WindowsChair()
    def create_table(self): return WindowsTable()

class MacFactory(UIFactory):
    def create_chair(self): return MacChair()
    def create_table(self): return MacTable()

# 사용자는 팩토리만 받고 나머지는 팩토리에 맡긴다
factory = WindowsFactory()
chair = factory.create_chair()
table = factory.create_table()

#### 데이터 시각화

#### Train 데이터 전처리

In [ ]:
# 1. 데이터 분할 (시계열이므로 shuffle=False)
train_data, test_data = train_test_split(df, test_size=0.2, shuffle=False)

# 2. Train 데이터에서 이상치를 제외한 'Clean' 데이터 생성
# (예: 상하위 1%를 제거하거나 RobustScaler의 기준이 되는 IQR 범위 내 데이터만 사용)
def clean_time_series_data(df, columns, multiplier=1.5):
    """
    df: 데이터프레임
    columns: 이상치를 체크할 컬럼 리스트 (예: ['f1', 'f2', 'f3'])
    multiplier: IQR 가중치 (보통 1.5, 더 엄격하게 잡으려면 1.0~1.2)
    """
    df_clean = df.copy()
    
    for col in columns:
        # 1. IQR 계산
        Q1 = df_clean[col].quantile(0.25)
        Q3 = df_clean[col].quantile(0.75)
        IQR = Q3 - Q1
        
        lower_bound = Q1 - multiplier * IQR
        upper_bound = Q3 + multiplier * IQR
        
        # 2. 범위를 벗어나는 데이터를 NaN으로 변경
        # (바로 삭제하지 않고 NaN으로 만드는 것이 포인트입니다)
        outlier_mask = (df_clean[col] < lower_bound) | (df_clean[col] > upper_bound)
        outlier_count = outlier_mask.sum()
        df_clean.loc[outlier_mask, col] = np.nan
        
        print(f"[{col}] 탐지된 이상치 개수: {outlier_count}")

    # 3. 보간(Interpolation) 수행
    # 'linear' 방식은 인접한 두 정상 데이터 사이를 직선으로 연결합니다.
    # 'limit_direction'은 양방향으로 채워줍니다.
    df_clean = df_clean.interpolate(method='linear', limit_direction='both')
    
    # 만약 첫 번째 행이나 마지막 행이 NaN이라 보간되지 않은 경우를 대비해 
    # 남은 NaN은 앞뒤 값으로 채웁니다.
    df_clean = df_clean.ffill().bfill()
    
    return df_clean

# --- 실전 적용 예시 ---
# 3개 피처를 가진 데이터프레임이라고 가정 (train_data)
feature_cols = train_data.columns.tolist() # 혹은 ['feat1', 'feat2', 'feat3']

# 이상치 보간 처리된 Clean 데이터 생성
train_clean = clean_time_series_data(train_data, feature_cols)

#### 데이터 스케일링

In [ ]:
# 1. 스케일러 선언 및 "Clean" 데이터로 학습
# RobustScaler는 그 자체로 이상치의 영향을 덜 받으므로 매우 적합합니다.
scaler = RobustScaler()

# 2. 3차원 -> 2차원으로 변환 (reshape)
# (Samples, Time_steps, Features) -> (Samples * Time_steps, Features)
train_reshaped = train_clean.reshape(-1, 3)

# 3. 학습 및 변환 (Transform)
# Train은 깨끗한 상태로 변환하여 학습에 사용
scaler.fit(train_reshaped) # 중요: 깨끗한 데이터의 통계값(중앙값, IQR)을 학습
train_scaled = scaler.transform(train_reshaped)

# Test는 이상치가 포함된 'Raw' 상태 그대로 변환
# (학습된 중앙값/IQR 기준으로 변환되므로 이상치는 매우 크거나 작은 값으로 남음)
test_scaled = scaler.transform(test_data)

normal_train_scaled = train_scaled.reshape(train_clean.shape)
normal_test_scaled = test_scaled.reshape(test.shape)

print("데이터 스케일링 완료")
print(f"Train mean: {normal_train_scaled.mean():.4f}, std: {normal_train_scaled.std():.4f}")

NameError: name 'RobustScaler' is not defined

#### Dataset 및 DataLoader 생성

In [ ]:
class TimeSeriesDataset(Dataset):
    def __init__(self, sequences):
        self.sequences = torch.FloatTensor(sequences)
    
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        return self.sequences[idx]

# Dataset 생성
train_dataset = TimeSeriesDataset(normal_train_scaled)
val_dataset = TimeSeriesDataset(normal_val_scaled)
test_dataset = TimeSeriesDataset(anomaly_test_scaled)

# DataLoader 생성
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"Train batches: {len(train_loader)}")
print(f"Val batches: {len(val_loader)}")
print(f"Test batches: {len(test_loader)}")

## 3. LSTM Autoencoder 모델 정의
LSTM Autoencoder는 Encoder-Decoder 구조로 이루어져 있습니다:  
 
Encoder: 입력 시퀀스를 잠재 표현(latent representation)으로 압축  
Decoder: 잠재 표현으로부터 원본 시퀀스를 재구성  

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, dropout, seq_len):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.seq_len = seq_len

        self.encoder_lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            dropout=dropout,
            batch_first=True
        )

    def forward(self, input_seq):
        # input_seq: (B, T, D)
        encoder_outputs, (h_n, c_n) = self.encoder_lstm(input_seq)

        # last hidden state -> latent representation
        latent = h_n.squeeze(0)  # (B, H)
        latent_seq = latent.unsqueeze(1).repeat(1, input_seq.size(1), 1)  # (B, T, H)

        return latent_seq, encoder_outputs


class Decoder(nn.Module):
    def __init__(self, output_dim, hidden_dim, dropout, seq_len, use_output_activation):
        super().__init__()
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.seq_len = seq_len
        self.use_output_activation = use_output_activation

        self.decoder_lstm = nn.LSTM(
            input_size=hidden_dim,
            hidden_size=hidden_dim,
            dropout=dropout,
            batch_first=True
        )

        self.output_layer = nn.Linear(hidden_dim, output_dim)
        self.activation = nn.Sigmoid()

    def forward(self, latent_seq):
        # latent_seq: (B, T, H)
        decoder_outputs, (h_n, c_n) = self.decoder_lstm(latent_seq)
        reconstructed_seq = self.output_layer(decoder_outputs)  # (B, T, D)

        if self.use_output_activation:
            reconstructed_seq = self.activation(reconstructed_seq)

        return reconstructed_seq, h_n


class LSTMAutoEncoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, dropout, seq_len, use_output_activation=False):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.seq_len = seq_len

        self.encoder = Encoder(
            input_dim=input_dim,
            hidden_dim=hidden_dim,
            dropout=dropout,
            seq_len=seq_len
        )

        self.decoder = Decoder(
            output_dim=input_dim,
            hidden_dim=hidden_dim,
            dropout=dropout,
            seq_len=seq_len,
            use_output_activation=use_output_activation
        )

    def forward(self, input_seq, return_latent=False, return_encoder_outputs=False):
        latent_seq, encoder_outputs = self.encoder(input_seq)
        reconstructed_seq, last_hidden = self.decoder(latent_seq)

        if return_latent:
            return reconstructed_seq, last_hidden
        elif return_encoder_outputs:
            return reconstructed_seq, encoder_outputs

        return reconstructed_seq

##### 모델 초기화

In [ ]:
# 하이퍼파라미터
input_dim = 3  # feature 개수
hidden_dim = 64  # LSTM hidden dimension
seq_len = 50  # sequence length
dropout = 0.2

# 모델 생성
model = LSTMAutoEncoder(
    input_dim=input_dim,
    hidden_dim=hidden_dim,
    dropout=dropout,
    seq_len=seq_len,
    use_output_activation=False
).to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

print(model)
print(f"\nTotal parameters: {sum(p.numel() for p in model.parameters())}")

## 4. 학습 함수 정의

In [ ]:
def train_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    
    for batch in dataloader:
        batch = batch.to(device)
        
        # Forward pass
        reconstructed = model(batch)
        loss = criterion(reconstructed, batch)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(dataloader)


def validate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    
    with torch.no_grad():
        for batch in dataloader:
            batch = batch.to(device)
            
            # Forward pass
            reconstructed = model(batch)
            loss = criterion(reconstructed, batch)
            
            total_loss += loss.item()
    
    return total_loss / len(dataloader)


def compute_reconstruction_error(model, dataloader, device):
    """
    각 샘플의 재구성 오차 계산 (이상 탐지용)
    """
    model.eval()
    errors = []
    
    with torch.no_grad():
        for batch in dataloader:
            batch = batch.to(device)
            reconstructed = model(batch)
            
            # 샘플별 MSE 계산
            batch_errors = torch.mean((batch - reconstructed) ** 2, dim=(1, 2))
            errors.extend(batch_errors.cpu().numpy())
    
    return np.array(errors)
    

## 5. 모델 학습

In [ ]:
# 학습 설정
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 50

# 학습 이력 저장
train_losses = []
val_losses = []

print("모델 학습 시작...\n")

for epoch in range(num_epochs):
    train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
    val_loss = validate(model, val_loader, criterion, device)
    
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}]")
        print(f"  Train Loss: {train_loss:.6f}")
        print(f"  Val Loss: {val_loss:.6f}")

print("\n학습 완료!")

#### 학습 곡선 시각화

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.title('LSTM Autoencoder Training Curves')
plt.legend()
plt.grid(True)
plt.show()

## 6. 이상 탐지 성능 평가

In [ ]:
# 재구성 오차 계산
print("재구성 오차 계산 중...")
normal_errors = compute_reconstruction_error(model, val_loader, device)
anomaly_errors = compute_reconstruction_error(model, test_loader, device)

print(f"정상 데이터 재구성 오차 - Mean: {normal_errors.mean():.6f}, Std: {normal_errors.std():.6f}")
print(f"이상 데이터 재구성 오차 - Mean: {anomaly_errors.mean():.6f}, Std: {anomaly_errors.std():.6f}")

#### 재구성 오차 분포 시각화

In [ ]:
plt.figure(figsize=(12, 5))

# 히스토그램
plt.subplot(1, 2, 1)
plt.hist(normal_errors, bins=30, alpha=0.7, label='Normal', color='blue')
plt.hist(anomaly_errors, bins=30, alpha=0.7, label='Anomaly', color='red')
plt.xlabel('Reconstruction Error')
plt.ylabel('Frequency')
plt.title('Reconstruction Error Distribution')
plt.legend()
plt.grid(True)

# 박스플롯
plt.subplot(1, 2, 2)
plt.boxplot([normal_errors, anomaly_errors], labels=['Normal', 'Anomaly'])
plt.ylabel('Reconstruction Error')
plt.title('Reconstruction Error Comparison')
plt.grid(True)

plt.tight_layout()
plt.show()

#### 임계값 기반 이상 탐지

In [ ]:
# 임계값 설정 (정상 데이터 평균 + 2*표준편차)
threshold = normal_errors.mean() + 2 * normal_errors.std()
print(f"이상 탐지 임계값: {threshold:.6f}")

# 이상 탐지 수행
normal_predictions = (normal_errors > threshold).astype(int)
anomaly_predictions = (anomaly_errors > threshold).astype(int)

# 성능 평가
normal_accuracy = 1 - normal_predictions.mean()
anomaly_accuracy = anomaly_predictions.mean()

print(f"\n정상 데이터 정확도: {normal_accuracy:.2%}")
print(f"이상 데이터 탐지율: {anomaly_accuracy:.2%}")

## 7. 재구성 결과 시각화

In [ ]:
# 정상 데이터 재구성 예시
model.eval()
with torch.no_grad():
    sample_normal = torch.FloatTensor(normal_val_scaled[0:1]).to(device)
    reconstructed_normal = model(sample_normal).cpu().numpy()[0]

    sample_anomaly = torch.FloatTensor(anomaly_test_scaled[0:1]).to(device)
    reconstructed_anomaly = model(sample_anomaly).cpu().numpy()[0]

# 시각화
fig, axes = plt.subplots(2, 3, figsize=(15, 8))

# 정상 데이터
for i in range(3):
    axes[0, i].plot(normal_val_scaled[0, :, i], label='Original', linewidth=2)
    axes[0, i].plot(reconstructed_normal[:, i], label='Reconstructed', linestyle='--', linewidth=2)
    axes[0, i].set_title(f'Normal - Feature {i+1}')
    axes[0, i].set_xlabel('Time')
    axes[0, i].set_ylabel('Value')
    axes[0, i].legend()
    axes[0, i].grid(True)

# 이상 데이터
for i in range(3):
    axes[1, i].plot(anomaly_test_scaled[0, :, i], label='Original', linewidth=2, color='red')
    axes[1, i].plot(reconstructed_anomaly[:, i], label='Reconstructed', linestyle='--', linewidth=2, color='orange')
    axes[1, i].set_title(f'Anomaly - Feature {i+1}')
    axes[1, i].set_xlabel('Time')
    axes[1, i].set_ylabel('Value')
    axes[1, i].legend()
    axes[1, i].grid(True)

plt.tight_layout()
plt.show()